# Simulated Correlated Noise in Quantum Error Correcting Circuits

In this notebook we provide two simple examples that demonstrate how to simulate correlated noise within a quantum error correcting circuit. To demonstrate this, we provide the key building block of the surface code, the $X$ and $Z$ check circuits. The results can be compared to Figure 3 in arxiv.org:xxxx.xxxxx. Unlike Fig. 3, this code only produces the SchWARMA plots. That is because the full Trotter simulations take very long. For the code that created those plots check the python scripts in the src subdirectory and the associated submit scripts that we used for our cluster.

This code makes use of the SchWARMA noise model available within the mezze package. This can be downloaded from (cite github source). In addition, we simulate the circuits using the open source package qutip. This can be downloaded [here](http://qutip.org/).

The standard $X$ and $Z$ check circuits for the surface code are:
![title](src/check_circuits.png)
We compile the CNOT into the circuit:
![title](src/compiled_cnot.png)
The gates are defined as: $$X=e^{-i \sigma_x \pi/2},$$ $$Y^{\pm 1/2} = e^{\mp i \sigma_y \pi/4},$$ $$Z^{1/2} = e^{-i\sigma_z \pi/4},$$ and the controlled $ZZ$ gate between qubits $i$ and $j$ is $$ZZ_{90} = e^{-i \sigma_z^{(i)}\otimes\sigma_z^{(j)} \pi/4}.$$ For the $Z$ check operator, all the single qubit $X$ and $Y$ rotations between the CNOT gates cancel each other. This allows for a further simplification of the compiled circuit of just single-qubit rotations on the ancilla to start, a sequence of $ZZ_{90}$ gates, followed by single qubit rotations on the ancilla. There are leftover virtual $Z^{1/2}$ gates to be done on the data qubits, but we leave these as they would be virtual and taken care of during the next round of error correction in any system.

In the simulations below, we compute unitary operators corresponding to the perfect circuits using qutip. We then simulate the exact same circuit, but we now intersperse random unitary rotations with the angle of rotation given by the SchWARMA model. This allows us to simulate time correlated noise within the circuit. The process fideli

In [ ]:
import sys
sys.path.append('./src')
import mezze
import mezze.metrics
import mezze.random.SchWARMA
from mezze.channel import _sigmaI, _sigmaX, _sigmaY, _sigmaZ
import numpy as np
from MultiAxisDephasingPMD import MultiAxisDephasing
from Gates import Gates
import pickle
import qecc
import qutip as qt
import time
import os
import scipy.signal as si
import scipy.linalg as la
from qutip_gates import *
import scipy.optimize as op
from collections import OrderedDict
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
%matplotlib inline

In [ ]:
# Noise spectra parameters to be run.
s_pow_list = [1.0e-12, 1.0e-10, 1e-8, 1e-6, 1e-4, 1e-2]
corr_time_list = [0.25, 0.5, 1.0, 2.0, 4.0, 8.0, 16.0, 32.0]
# The following two numbers specify the number of discrete time steps in the quantum circuit.
# The X and Z check circuits are slightly different depth as the Z check compilation allows some 
# intermediate gates to be cancelled
NN_specX = 24 
NN_specZ = 18
# Set the number of Monte Carlo Samples to be used. 1000 samples were used for the plots in the paper.
# 50 samples provides relatively decent convergence. It takes about 10 minutes to run each plot on a laptop.
num_realz = 50 

In [ ]:
# Setup containers to hold plots
xcorr_plot_schwarma_inf = OrderedDict()
xcorr_plot_schwarma_inf_stdev = OrderedDict()
for el in s_pow_list:
    xcorr_plot_schwarma_inf[el] = []
    xcorr_plot_schwarma_inf_stdev[el] = []

schwarma_start = time.time()
for amp_idx in range(len(s_pow_list)):
    for corr_idx in range(len(corr_time_list)):
        s_pow = s_pow_list[amp_idx]
        s_amp = np.sqrt(s_pow)
        corr_time = corr_time_list[corr_idx]

        # We now setup a SchWARMA model using the built in helper functions for a Gaussian moving average model
        # Get Schwarma model parameters using build in helper functions. We note that this notebook gives slightly 
        # different values than those used in the paper. In the paper, we generate noise using our high-fidelity
        # Trotter based simulation, then compute the SchWARMA model that fits the Trotter model. He was just generate
        # the SchWARMA model directly. Noise is set equal on all three axes.
        a = [1,]
        bx = mezze.random.SchWARMA.gaussian_ma(corr_time)
        by = mezze.random.SchWARMA.gaussian_ma(corr_time)
        bz = mezze.random.SchWARMA.gaussian_ma(corr_time)

        # Generate perfect X syndrome circuit using qutip
        qx = qt.qip.QubitCircuit(5, reverse_states=False)
        qx.add_gate("SNOT", targets=[0])
        qx.add_gate("CNOT", controls=[0], targets=[2])
        qx.add_gate("CNOT", controls=[0], targets=[1])
        qx.add_gate("CNOT", controls=[0], targets=[4])
        qx.add_gate("CNOT", controls=[0], targets=[3])
        qx.add_gate("SNOT", targets=[0])
        perfect_xcircuit = qt.gate_sequence_product(qx.propagators())

        # Now simulate syndrome circuit with SchWARMA noise model and qutip circuit simulator
        schwarma_xcircuit_infidelities = [0.0]*num_realz
        for k in range(num_realz):
            # Generate the angles to be used for the noise amplitudes in the circuit. For x,y,and,z rotations
            # a discrete set of NN_specX angles are generated. These are used in a noise model that causes a 
            # rotation of expm(-0.5j*(schwarma_xangles[k]*sigma_x + schwarma_yangles[k]*sigma_y + schwarma_zangles[k]*sigma_z))
            # at the discrete location k within the circuit.
            schwarma_xangles = np.array([mezze.random.SchWARMA.schwarma_trajectory(NN_specX, a, bx, amp=s_amp) for i in range(5)])
            schwarma_yangles = np.array([mezze.random.SchWARMA.schwarma_trajectory(NN_specX, a, by, amp=s_amp) for i in range(5)])
            schwarma_zangles = np.array([mezze.random.SchWARMA.schwarma_trajectory(NN_specX, a, bz, amp=s_amp) for i in range(5)])
            # Setup the qutip circuit
            qx_err = qt.qip.QubitCircuit(5, reverse_states=False)
            # Set up user defined gates for the ZZ interaction and the xyz noise model
            qx_err.user_gates = {"CZZ": zz_gate, "XYZ": xyz_gate}
            # Generate circuit with noise
            qx_err.add_gate("SNOT", targets=[0])
            # Helper functions are defined in qutip_gates.py. They add in the CNOT gates as well as the noise
            # using the compiled circuit defined above. For each CNOT, we pass in just the angles needed for that segment
            # of the circuit.
            add_qtcnot(qx_err, 0, 2, xerr=schwarma_xangles[:, 0:NN_specX//4], yerr=schwarma_yangles[:, 0:NN_specX//4], zerr=schwarma_zangles[:, 0:NN_specX//4])
            add_qtcnot(qx_err, 0, 1, xerr=schwarma_xangles[:, NN_specX//4:2*NN_specX//4], yerr=schwarma_yangles[:, NN_specX//4:2*NN_specX//4], zerr=schwarma_zangles[:, NN_specX//4:2*NN_specX//4])
            add_qtcnot(qx_err, 0, 4, xerr=schwarma_xangles[:, 2*NN_specX//4:3*NN_specX//4], yerr=schwarma_yangles[:, 2*NN_specX//4:3*NN_specX//4], zerr=schwarma_zangles[:, 2*NN_specX//4:3*NN_specX//4])
            add_qtcnot(qx_err, 0, 3, xerr=schwarma_xangles[:, 3*NN_specX//4:NN_specX], yerr=schwarma_yangles[:, 3*NN_specX//4:NN_specX], zerr=schwarma_zangles[:, 3*NN_specX//4:NN_specX])
            qx_err.add_gate("SNOT", targets=[0])
            noisy_schwarma_xcircuit = qt.gate_sequence_product(qx_err.propagators())
            # Store the circuit fidelity for the circuit relative to a perfect circuit
            schwarma_xcircuit_infidelities[k] = mezze.metrics.process_infidelity(mezze.channel.QuantumChannel(noisy_schwarma_xcircuit.full(), 'unitary'), mezze.channel.QuantumChannel(perfect_xcircuit.full(), 'unitary'))

        xcorr_plot_schwarma_inf[s_pow].append(np.mean(schwarma_xcircuit_infidelities))
        xcorr_plot_schwarma_inf_stdev[s_pow].append(np.std(schwarma_xcircuit_infidelities))
    
    print (f'{100*(amp_idx+1)/len(s_pow_list):3.1f}% done')
        
schwarma_end = time.time()
print(f'X SchWARMA runtime: {schwarma_end - schwarma_start:.1f} seconds')

In [ ]:
colors = cm.rainbow(np.linspace(0, 1, len(xcorr_plot_schwarma_inf)))
c=0
for key in xcorr_plot_schwarma_inf.keys():
    y_data_schwarma = xcorr_plot_schwarma_inf[key]
    y_err_schwarma = 0.434 * np.array(xcorr_plot_schwarma_inf_stdev[key]) / np.array(xcorr_plot_schwarma_inf[key])
    plt.xscale("log", nonposx='clip', basex=2)
    plt.errorbar(corr_time_list, np.log10(y_data_schwarma), yerr = y_err_schwarma, color=colors[c], marker='p', label=str(key)+str(" SchWARMA"), capsize=3)
    c=c+1

ylim = plt.ylim()
plt.ylim([np.min(ylim), 0])
plt.title("X Syndrome")
plt.xlabel('Noise Correlation Time (Gate Lengths)')
plt.ylabel('Infidelity')
locs, labels = plt.yticks()
plt.yticks(locs, ['$10^{'+str(int(loc))+'}$' for loc in locs])
plt.tight_layout()
plt.show()

In [ ]:
# Setup containers to hold plots
zcorr_plot_schwarma_inf = OrderedDict()
zcorr_plot_schwarma_inf_stdev = OrderedDict()
for el in s_pow_list:
    zcorr_plot_schwarma_inf[el] = []
    zcorr_plot_schwarma_inf_stdev[el] = []

schwarma_start = time.time()
for amp_idx in range(len(s_pow_list)):
    for corr_idx in range(len(corr_time_list)):
        s_pow = s_pow_list[amp_idx]
        s_amp = np.sqrt(s_pow)
        corr_time = corr_time_list[corr_idx]

        # We now setup a SchWARMA model using the built in helper functions for a Gaussian moving average model
        # Get Schwarma model parameters using build in helper functions. We note that this notebook gives slightly 
        # different values than those used in the paper. In the paper, we generate noise using our high-fidelity
        # Trotter based simulation, then compute the SchWARMA model that fits the Trotter model. He was just generate
        # the SchWARMA model directly. Noise is set equal on all three axes.
        a = [1,]
        bx = mezze.random.SchWARMA.gaussian_ma(corr_time)
        by = mezze.random.SchWARMA.gaussian_ma(corr_time)
        bz = mezze.random.SchWARMA.gaussian_ma(corr_time)

        # Generate perfect Z syndrome circuit using qutip
        qz = qt.qip.QubitCircuit(5, reverse_states=False)
        qz.user_gates = {"CZZ": zz_gate}
        qz.add_gate("SNOT", targets=[0])
        qz.add_gate("CZZ", arg_value=-np.pi / 2.0, targets=[2, 0])
        qz.add_gate("CZZ", arg_value=-np.pi / 2.0, targets=[1, 0])
        qz.add_gate("CZZ", arg_value=-np.pi / 2.0, targets=[4, 0])
        qz.add_gate("CZZ", arg_value=-np.pi / 2.0, targets=[3, 0])
        qz.add_gate("SNOT", targets=[0])
        perfect_zcircuit = qt.gate_sequence_product(qz.propagators())
        
        # Now simulate syndrome circuit with SchWARMA noise model and qutip circuit simulator
        schwarma_zcircuit_infidelities = [0.0]*num_realz
        for k in range(num_realz):
            # Generate the angles to be used for the noise amplitudes in the circuit. For x,y,and,z rotations
            # a discrete set of NN_specX angles are generated. These are used in a noise model that causes a 
            # rotation of expm(-0.5j*(schwarma_xangles[k]*sigma_x + schwarma_yangles[k]*sigma_y + schwarma_zangles[k]*sigma_z))
            # at the discrete location k within the circuit.
            schwarma_xangles = np.array([mezze.random.SchWARMA.schwarma_trajectory(NN_specZ, a, bx, amp=s_amp) for i in range(5)])
            schwarma_yangles = np.array([mezze.random.SchWARMA.schwarma_trajectory(NN_specZ, a, by, amp=s_amp) for i in range(5)])
            schwarma_zangles = np.array([mezze.random.SchWARMA.schwarma_trajectory(NN_specZ, a, bz, amp=s_amp) for i in range(5)])
            # Setup the qutip circuit
            qz_err = qt.qip.QubitCircuit(5, reverse_states=False)
            # Set up user defined gates for the ZZ interaction and the xyz noise model
            qz_err.user_gates = {"CZZ": zz_gate, "XYZ": xyz_gate}
            # Helper functions are defined in qutip_gates.py. They add in the CZ gates as well as the noise
            # using the compiled circuit defined above. For each CNOT, we pass in just the angles needed for that segment
            # of the circuit.
            add_qth(qz_err, 0, xerr=schwarma_xangles[:, 0], yerr=schwarma_yangles[:, 0], zerr=schwarma_zangles[:, 0])
            add_qtzz(qz_err, 2, 0, xerr=schwarma_xangles[:, 1:1+NN_specZ // 4], yerr=schwarma_yangles[:, 1:1+NN_specZ // 4], zerr=schwarma_zangles[:, 1:1+NN_specZ // 4])
            add_qtzz(qz_err, 1, 0, xerr=schwarma_xangles[:, 1+NN_specZ // 4:1+2 * NN_specZ // 4], yerr=schwarma_yangles[:, 1+NN_specZ // 4:1+2 * NN_specZ // 4], zerr=schwarma_zangles[:, 1+NN_specZ // 4:1+2 * NN_specZ // 4])
            add_qtzz(qz_err, 4, 0, xerr=schwarma_xangles[:, 1+2 * NN_specZ // 4:1+3 * NN_specZ // 4], yerr=schwarma_yangles[:, 1+2 * NN_specZ // 4:1+3 * NN_specZ // 4], zerr=schwarma_zangles[:, 1+2 * NN_specZ // 4:1+3 * NN_specZ // 4])
            add_qtzz(qz_err, 3, 0, xerr=schwarma_xangles[:, 1+3 * NN_specZ // 4:1+4*NN_specZ//4], yerr=schwarma_yangles[:, 1+3 * NN_specZ // 4:1+4*NN_specZ//4], zerr=schwarma_zangles[:, 1+3 * NN_specZ // 4:1+4*NN_specZ//4])
            add_qth(qz_err, 0, xerr=schwarma_xangles[:, NN_specZ -1], yerr=schwarma_yangles[:, NN_specZ -1], zerr=schwarma_zangles[:, NN_specZ -1])
            noisy_schwarma_xcircuit = qt.gate_sequence_product(qz_err.propagators())
            # Store the circuit fidelity for the circuit relative to a perfect circuit
            schwarma_zcircuit_infidelities[k] = mezze.metrics.process_infidelity(mezze.channel.QuantumChannel(noisy_schwarma_xcircuit.full(), 'unitary'), mezze.channel.QuantumChannel(perfect_zcircuit.full(), 'unitary'))

        zcorr_plot_schwarma_inf[s_pow].append(np.mean(schwarma_zcircuit_infidelities))
        zcorr_plot_schwarma_inf_stdev[s_pow].append(np.std(schwarma_zcircuit_infidelities))
        
    print (f'{100*(amp_idx+1)/len(s_pow_list):3.1f}% done')

schwarma_end = time.time()
print(f'Z SchWARMA runtime: {schwarma_end - schwarma_start:.1f} seconds')

In [ ]:
colors = cm.rainbow(np.linspace(0, 1, len(zcorr_plot_schwarma_inf)))
c=0
for key in zcorr_plot_schwarma_inf.keys():
    y_data_schwarma = zcorr_plot_schwarma_inf[key]
    y_err_schwarma = 0.434 * np.array(zcorr_plot_schwarma_inf_stdev[key]) / np.array(zcorr_plot_schwarma_inf[key])
    plt.xscale("log", nonposx='clip', basex=2)
    plt.errorbar(corr_time_list, np.log10(y_data_schwarma), yerr = y_err_schwarma, color=colors[c], marker='p', label=str(key)+str(" SchWARMA"), capsize=3)
    c=c+1

ylim = plt.ylim()
plt.ylim([np.min(ylim), 0])
plt.title("Z Syndrome")
plt.xlabel('Noise Correlation Time (Gate Lengths)')
plt.ylabel('Infidelity')
locs, labels = plt.yticks()
plt.yticks(locs, ['$10^{'+str(int(loc))+'}$' for loc in locs])
plt.tight_layout()
plt.show()